In [4]:
# All necessary libraries must be put here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd

In [5]:
# Read the CRSP Data and make it become a dataframe
crsp_data_1993_2019 = pd.read_csv("/Users/namnguyen/Desktop/Research/Don/Imaging_Price_Trends_Code/crsp_data_20240719_1.csv")

# Check the data
crsp_data_1993_2019.head(5)

/var/folders/pl/z1lfm5xs7299k23shpwd6ngh0000gp/T/ipykernel_22954/4244800754.py:2: DtypeWarning:

Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.



,PERMNO,HdrCUSIP,Ticker,PERMCO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen
0,10001,36720410,GFGC,7953,1993-01-04,150.0,14.5,14.50,14.5,14.50
1,10001,36720410,GFGC,7953,1993-01-05,0.0,NaN,NaN,NaN,NaN
2,10001,36720410,GFGC,7953,1993-01-06,0.0,NaN,NaN,NaN,NaN
3,10001,36720410,GFGC,7953,1993-01-07,228.0,14.5,14.50,14.5,14.50
4,10001,36720410,GFGC,7953,1993-01-08,1375.0,14.5,14.25,14.5,14.25


In [19]:
### DRAFT VERSION ON A RANDOM SET OF DATA
### CAN IGNORE THIS PART
# Sample data creation
dates = pd.date_range(start = '2023-01-01', periods = 20)
data = {
    "Date": dates,
    "Open": np.random.uniform(100, 200, size = 20),
    "High": np.random.uniform(200, 300, size = 20),
    "Low": np.random.uniform(50, 100, size = 20),
    "Close": np.random.uniform(100, 250, size = 20),
    "Volume": np.random.randint(1000, 5000, size = 20)
}

df = pd.DataFrame(data, index = pd.DatetimeIndex(data["Date"]))

# Calculate the 20-day moving average
df["20_MA"] = df["Close"].rolling(window = 20).mean()

# Create figure and axis
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (60, 64), gridspec_kw = {"height_ratios": [4, 1]})

# Plot OHLC
ax1.plot(df["Date"], df["Close"], label = "Close", color = "white")
ax1.plot(df["Date"], df["20_MA"], label = "20-Day MA", color = "white", linestyle = '--')
mpf.plot(df, type = "candle", ax = ax1, volume = False, style = 'charles', show_nontrading = False)
ax1.set_title('OHLC Chart with 20-Day Moving Average')
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax1.xaxis.set_major_locator(mdates.DayLocator())
ax1.set_ylabel('Price')
ax1.legend()

# Plot Volume
ax2.bar(df["Date"], df["Volume"], color = "white")

# Set background color to black
fig.patch.set_color("black")
ax1.set_facecolor("black")
ax2.set_facecolor("black")

# Save as black and white image
plt.subplots_adjust(hspace=0.5)
plt.savefig(f'ohlc_volume_chart_20_days.png', dpi=300, bbox_inches = 'tight')
plt.show()



Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to genera

KeyboardInterrupt: 

Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 21315 ticks ([-969.0, ..., 20345.0]), which exceeds Locator.MAXTICKS (1000).


In [ ]:
### WE REPLICATE THE IMAGE AGAIN TO CHECK THE FOLLOWING REQUIREMENT:
# 1. IS THE IMAGE SPARSE (MEANS THAT MOST OF PIXELS ARE 0)?
# 2. IS THAT "REALLY" BLACK AND WHITE (0 AND 255), OR UNEXPECTEDLY HAS SOME OTHER NUMBERS?
# 3. DOES THE MOVING AVERAGE SATISFY THE AUTHORS' REQUIREMENTS, WHICH IS STAIRS-STYLE?

In [ ]:
# CHECK THE AMOUNT OF PERMNO THAT WE HAVE
unique_id = crsp_data_1993_2019["PERMNO"].unique()
print(unique_id)
print(f"Len of unique ID: {len(unique_id)}")